In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [4]:
# Step 1: Upload the file
from google.colab import files
uploaded = files.upload()

# Step 2: Load the data into a DataFrame
import pandas as pd

# Assuming the uploaded file is 'Books.csv'
df = pd.read_csv('Books.csv')
df.head()



Saving Books.csv to Books.csv


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Unnamed: 12
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,9.78044E+12,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.78044E+12,eng,870,2153167,29221,09-01-2004,Scholastic Inc.,NaN
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.78044E+12,eng,352,6333,244,11-01-2003,Scholastic,NaN
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9.78044E+12,eng,435,2339585,36325,05-01-2004,Scholastic Inc.,NaN
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,439682584,9.78044E+12,eng,2690,41428,164,9/13/2004,Scholastic,NaN


In [10]:
def search_books(keyword):
    """
    Search for books that match the keyword in the title, authors, or publisher.
    """
    keyword = keyword.lower()
    results = df[
        df['title'].str.lower().str.contains(keyword) |
        df['authors'].str.lower().str.contains(keyword) |
        df['publisher'].str.lower().str.contains(keyword)
    ]
    return results


def display_books(books):
    """
    Display the list of books.
    """
    if books.empty:
        print("No books found.")
    else:
        for index, book in books.iterrows():
            print(f"Title: {book['title']}")
            print(f"Author: {book['authors']}")
            print(f"Average Rating: {book['average_rating']}")
            print(f"ISBN: {book['isbn']}")
            print(f"ISBN13: {book['isbn13']}")
            print(f"Language: {book['language_code']}")
            # Check if 'num_pages' column exists before printing
            if 'num_pages' in book:
                print(f"Number of Pages: {book['num_pages']}")
            print(f"Ratings Count: {book['ratings_count']}")
            print(f"Text Reviews Count: {book['text_reviews_count']}")
            print(f"Publication Date: {book['publication_date']}")
            print(f"Publisher: {book['publisher']}")
            print('-' * 40)

def main():
    print("Welcome to the Book Recommendation Program!")
    keyword = input("Enter a keyword to search for books: ")
    results = search_books(keyword)
    display_books(results)

if __name__ == "__main__":
    main()

Welcome to the Book Recommendation Program!
Enter a keyword to search for books: William T. Vollmann
Title: Journey to the End of the Night
Author: Louis-Ferdinand Céline/Ralph Manheim/William T. Vollmann
Average Rating: 4.23
ISBN: 811216543
ISBN13: 9.78081E+12
Language: eng
Ratings Count: 21561
Text Reviews Count: 876
Publication Date: 5/17/2006
Publisher: New Directions
----------------------------------------
Title: Europe Central
Author: William T. Vollmann
Average Rating: 3.92
ISBN: 143036599
ISBN13: 9.78014E+12
Language: en-US
Ratings Count: 2121
Text Reviews Count: 247
Publication Date: 12-01-2005
Publisher: Penguin
----------------------------------------
Title: Dirty Snow
Author: Georges Simenon/Marc Romano/William T. Vollmann
Average Rating: 3.93
ISBN: 1590170431
ISBN13: 9.78159E+12
Language: eng
Ratings Count: 1780
Text Reviews Count: 220
Publication Date: 8/31/2003
Publisher: NYRB Classics
----------------------------------------
Title: Whores for Gloria
Author: William T. 

In [13]:
from google.colab import files
uploaded = files.upload()


Saving Books.csv to Books (1).csv


In [18]:

df.fillna('', inplace=True)

# Step 4: Combine relevant text columns into a single feature vector for TF-IDF
df['combined_features'] = df['title'] + ' ' + df['authors']

# Step 5: Implement TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

# Step 6: Compute Similarity Matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Step 7: Define Recommendation Function
def recommend_books(title, cosine_sim=cosine_sim, books_df=df, num_recommendations=10):
    # Get the index of the book that matches the title
    idx = df.index[df['title'].str.lower() == title.lower()].tolist()[0]

    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:num_recommendations+1]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    return books_df.iloc[book_indices][['title', 'authors', 'average_rating']]




In [20]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined_features'])


In [21]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [29]:
def recommend_books(title, cosine_sim=cosine_sim, df=df, num_recommendations=10):
    # Convert the title to lowercase for case-insensitive matching
    title = title.lower()

    # Check if the title exists in the dataset
    if title not in df['title'].str.lower().tolist():
        return "Title not found in the dataset. Please check your input."

    # Get the index of the book that matches the title
    idx = df.index[df['title'].str.lower() == title].tolist()[0]

    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:num_recommendations+1]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    return df.iloc[book_indices][['title', 'authors', 'average_rating']]


In [31]:
recommended_books = recommend_books('The Lost Continent: Travels in Small Town America')
print(recommended_books)

                                                   title  \
2834   The Lost Continent:  Travels in Small-town Ame...   
18             Neither Here nor There: Travels in Europe   
19                             Notes from a Small Island   
10522                                         Down Under   
7664                                  The Lost Continent   
10526                    Bill Bryson: The Complete Notes   
13                           Bill Bryson's African Diary   
5555                         Bill Bryson's African Diary   
1545   The Innocent Man: Murder and Injustice in a Sm...   
7687          Someone Comes to Town  Someone Leaves Town   

                    authors average_rating  
2834            Bill Bryson           3.83  
18              Bill Bryson           3.86  
19              Bill Bryson           3.91  
10522           Bill Bryson           4.07  
7664   Edgar Rice Burroughs           3.68  
10526           Bill Bryson           4.09  
13              Bill Bry